# Airline Crew Pairing Optimization
## Set Partitioning Problem using Mixed-Integer Linear Programming (MILP)

**Course:** MATH F212 - Integer Programming

**Problem Overview:**
Optimize crew assignments for an airline by selecting the minimum-cost subset of crew pairings such that every flight is covered exactly once.

**Mathematical Formulation:**
- **Objective:** Minimize Z = Σ(j=1 to N) c_j * x_j
- **Constraints:** Σ(j=1 to N) a_{ij} * x_j = 1 for each flight i
- **Decision Variables:** x_j ∈ {0,1}

In [1]:
# Cell 1: Imports and Setup
import numpy as np
import pandas as pd
import pulp
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpStatus, value, PULP_CBC_CMD
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import random
from itertools import combinations, permutations
import time
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)

print("All imports successful!")
print(f"PuLP available: {hasattr(pulp, 'LpProblem')}")

All imports successful!
PuLP available: True


In [2]:
# Cell 2: Define Airports and Flight Data Structures

# Major hub airports with coordinates (lat, lon) and timezone offset
AIRPORTS = {
    'ATL': {'name': 'Atlanta', 'lat': 33.6407, 'lon': -84.4277, 'tz': -5, 'is_base': True},
    'ORD': {'name': 'Chicago', 'lat': 41.9742, 'lon': -87.9073, 'tz': -6, 'is_base': True},
    'DFW': {'name': 'Dallas', 'lat': 32.8998, 'lon': -97.0403, 'tz': -6, 'is_base': True},
    'DEN': {'name': 'Denver', 'lat': 39.8561, 'lon': -104.6737, 'tz': -7, 'is_base': False},
    'LAX': {'name': 'Los Angeles', 'lat': 33.9416, 'lon': -118.4085, 'tz': -8, 'is_base': True},
    'JFK': {'name': 'New York JFK', 'lat': 40.6413, 'lon': -73.7781, 'tz': -5, 'is_base': True},
    'SFO': {'name': 'San Francisco', 'lat': 37.6213, 'lon': -122.379, 'tz': -8, 'is_base': False},
    'SEA': {'name': 'Seattle', 'lat': 47.4502, 'lon': -122.3088, 'tz': -8, 'is_base': False},
    'MIA': {'name': 'Miami', 'lat': 25.7959, 'lon': -80.2870, 'tz': -5, 'is_base': False},
    'PHX': {'name': 'Phoenix', 'lat': 33.4373, 'lon': -112.0078, 'tz': -7, 'is_base': False}
}

# Get crew bases (airports where crews can start/end)
CREW_BASES = [code for code, info in AIRPORTS.items() if info['is_base']]

print(f"Total airports: {len(AIRPORTS)}")
print(f"Crew bases: {CREW_BASES}")
print(f"\nAirport details:")
for code, info in AIRPORTS.items():
    base_str = ' (CREW BASE)' if info['is_base'] else ''
    print(f"  {code}: {info['name']}{base_str}")

Total airports: 10
Crew bases: ['ATL', 'ORD', 'DFW', 'LAX', 'JFK']

Airport details:
  ATL: Atlanta (CREW BASE)
  ORD: Chicago (CREW BASE)
  DFW: Dallas (CREW BASE)
  DEN: Denver
  LAX: Los Angeles (CREW BASE)
  JFK: New York JFK (CREW BASE)
  SFO: San Francisco
  SEA: Seattle
  MIA: Miami
  PHX: Phoenix


In [3]:
# Cell 3: Flight Generation Function
import math

def calculate_flight_time(origin, destination):
    """Calculate flight time based on great circle distance."""
    lat1, lon1 = math.radians(AIRPORTS[origin]['lat']), math.radians(AIRPORTS[origin]['lon'])
    lat2, lon2 = math.radians(AIRPORTS[destination]['lat']), math.radians(AIRPORTS[destination]['lon'])
    
    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))
    distance_km = 6371 * c  # Earth radius in km
    
    # Average speed 800 km/h + 30 min for takeoff/landing
    flight_hours = distance_km / 800 + 0.5
    return max(1.0, min(6.0, flight_hours))  # Clamp between 1-6 hours

def generate_flights(num_flights, seed=42):
    """Generate realistic flight legs."""
    random.seed(seed)
    np.random.seed(seed)
    
    flights = []
    airport_codes = list(AIRPORTS.keys())
    base_date = datetime(2024, 1, 15, 6, 0)  # Start at 6 AM
    
    for i in range(num_flights):
        # Select origin and destination (different airports)
        origin = random.choice(airport_codes)
        destination = random.choice([a for a in airport_codes if a != origin])
        
        # Calculate flight duration
        duration_hours = calculate_flight_time(origin, destination)
        duration_minutes = int(duration_hours * 60)
        
        # Random departure time throughout the day (6 AM to 10 PM)
        dep_hour = random.randint(6, 22)
        dep_minute = random.choice([0, 15, 30, 45])
        departure = base_date.replace(hour=dep_hour, minute=dep_minute)
        arrival = departure + timedelta(minutes=duration_minutes)
        
        flights.append({
            'flight_id': i,
            'flight_number': f'FL{1000 + i}',
            'origin': origin,
            'destination': destination,
            'departure': departure,
            'arrival': arrival,
            'duration_hours': round(duration_hours, 2)
        })
    
    return pd.DataFrame(flights)

# Test the function
test_flights = generate_flights(5)
print("Sample generated flights:")
print(test_flights.to_string())

Sample generated flights:
   flight_id flight_number origin destination           departure             arrival  duration_hours
0          0        FL1000    ORD         ATL 2024-01-15 14:15:00 2024-01-15 15:58:00            1.72
1          1        FL1001    DEN         DFW 2024-01-15 09:00:00 2024-01-15 10:47:00            1.79
2          2        FL1002    PHX         SFO 2024-01-15 07:00:00 2024-01-15 08:48:00            1.81
3          3        FL1003    ORD         LAX 2024-01-15 13:00:00 2024-01-15 17:00:00            4.00
4          4        FL1004    MIA         DEN 2024-01-15 19:15:00 2024-01-15 23:11:00            3.94


In [4]:
# Cell 4: Crew Pairing Generation Function

def generate_pairings(flights_df, num_pairings=200, seed=42):
    """
    Generate legal crew pairings with FAA constraints:
    - Max 8 hours flying per duty day
    - Minimum 30 min connection time
    - Pairing starts and ends at same crew base
    """
    random.seed(seed)
    np.random.seed(seed)
    
    pairings = []
    flight_list = flights_df.to_dict('records')
    pairing_id = 0
    
    # Sort flights by departure time
    sorted_flights = sorted(flight_list, key=lambda x: x['departure'])
    
    attempts = 0
    max_attempts = num_pairings * 20
    
    while len(pairings) < num_pairings and attempts < max_attempts:
        attempts += 1
        
        # Pick a random crew base
        base = random.choice(CREW_BASES)
        
        # Find flights departing from this base
        base_departures = [f for f in sorted_flights if f['origin'] == base]
        if not base_departures:
            continue
        
        # Start building a pairing
        current_flight = random.choice(base_departures)
        pairing_flights = [current_flight]
        total_flying_hours = current_flight['duration_hours']
        current_location = current_flight['destination']
        current_time = current_flight['arrival']
        
        # Try to add more flights (1-4 legs total)
        max_legs = random.randint(1, 4)
        
        for _ in range(max_legs - 1):
            # Find connecting flights
            min_connection = timedelta(minutes=30)  # Minimum connection time
            max_connection = timedelta(hours=4)  # Maximum wait time
            
            connecting = [
                f for f in sorted_flights
                if f['origin'] == current_location
                and f['departure'] >= current_time + min_connection
                and f['departure'] <= current_time + max_connection
                and f['flight_id'] not in [pf['flight_id'] for pf in pairing_flights]
                and total_flying_hours + f['duration_hours'] <= 8.0  # FAA max
            ]
            
            if not connecting:
                break
            
            next_flight = random.choice(connecting)
            pairing_flights.append(next_flight)
            total_flying_hours += next_flight['duration_hours']
            current_location = next_flight['destination']
            current_time = next_flight['arrival']
        
        # Check if pairing ends at crew base (same base or any base)
        ends_at_base = current_location in CREW_BASES
        
        if ends_at_base and len(pairing_flights) >= 1:
            # Calculate pairing cost
            flight_ids = [f['flight_id'] for f in pairing_flights]
            
            # Cost components: base salary + per hour + per leg + layover
            base_cost = 200
            hourly_cost = total_flying_hours * 50
            leg_cost = len(pairing_flights) * 25
            layover_cost = 100 if current_location != base else 0
            total_cost = base_cost + hourly_cost + leg_cost + layover_cost
            
            pairings.append({
                'pairing_id': pairing_id,
                'flight_ids': flight_ids,
                'num_legs': len(pairing_flights),
                'base': base,
                'end_location': current_location,
                'total_flying_hours': round(total_flying_hours, 2),
                'cost': round(total_cost, 2)
            })
            pairing_id += 1
    
    return pairings

# Test pairing generation
test_flights = generate_flights(20, seed=42)
test_pairings = generate_pairings(test_flights, num_pairings=30, seed=42)
print(f"Generated {len(test_pairings)} pairings from {len(test_flights)} flights")
print("\nSample pairings:")
for p in test_pairings[:5]:
    print(f"  Pairing {p['pairing_id']}: Flights {p['flight_ids']}, {p['num_legs']} legs, "
          f"{p['total_flying_hours']}h, ${p['cost']}")

Generated 30 pairings from 20 flights

Sample pairings:
  Pairing 0: Flights [3], 1 legs, 4.0h, $525.0
  Pairing 1: Flights [3], 1 legs, 4.0h, $525.0
  Pairing 2: Flights [12], 1 legs, 1.72h, $411.0
  Pairing 3: Flights [12], 1 legs, 1.72h, $411.0
  Pairing 4: Flights [3], 1 legs, 4.0h, $525.0


In [5]:
# Cell 5: Build Constraint Matrix A

def build_constraint_matrix(flights_df, pairings):
    """
    Build the constraint matrix A where:
    a_{ij} = 1 if flight i is covered by pairing j, 0 otherwise
    """
    num_flights = len(flights_df)
    num_pairings = len(pairings)
    
    # Initialize constraint matrix
    A = np.zeros((num_flights, num_pairings), dtype=int)
    
    for j, pairing in enumerate(pairings):
        for flight_id in pairing['flight_ids']:
            A[flight_id, j] = 1
    
    # Get costs vector
    costs = np.array([p['cost'] for p in pairings])
    
    return A, costs

# Test constraint matrix
A_test, costs_test = build_constraint_matrix(test_flights, test_pairings)
print(f"Constraint matrix shape: {A_test.shape}")
print(f"(M flights x N pairings): ({len(test_flights)} x {len(test_pairings)})")
print(f"\nFlight coverage count (how many pairings cover each flight):")
coverage = A_test.sum(axis=1)
print(f"  Min coverage: {coverage.min()}, Max: {coverage.max()}, Mean: {coverage.mean():.2f}")
print(f"\nFlights with zero coverage: {(coverage == 0).sum()}")

Constraint matrix shape: (20, 30)
(M flights x N pairings): (20 x 30)

Flight coverage count (how many pairings cover each flight):
  Min coverage: 0, Max: 10, Mean: 1.50

Flights with zero coverage: 15


In [6]:
# Cell 6: MILP Solver for Set Partitioning Problem

def solve_crew_pairing(flights_df, pairings, A, costs, time_limit=300):
    """
    Solve the Set Partitioning Problem using MILP.
    
    Minimize: sum(c_j * x_j) for all pairings j
    Subject to: sum(a_ij * x_j) = 1 for all flights i
                x_j in {0, 1}
    """
    num_flights, num_pairings = A.shape
    
    # Create the problem
    prob = LpProblem("Crew_Pairing_Optimization", LpMinimize)
    
    # Decision variables: x_j = 1 if pairing j is selected
    x = [LpVariable(f"x_{j}", cat='Binary') for j in range(num_pairings)]
    
    # Objective: Minimize total cost
    prob += lpSum([costs[j] * x[j] for j in range(num_pairings)]), "Total_Cost"
    
    # Constraints: Each flight must be covered exactly once
    for i in range(num_flights):
        prob += lpSum([A[i, j] * x[j] for j in range(num_pairings)]) == 1, f"Flight_{i}_Coverage"
    
    # Solve
    start_time = time.time()
    prob.solve(PULP_CBC_CMD(msg=0, timeLimit=time_limit))
    solve_time = time.time() - start_time
    
    # Extract results
    status = LpStatus[prob.status]
    
    if status == 'Optimal':
        selected_pairings = [j for j in range(num_pairings) if value(x[j]) == 1]
        total_cost = value(prob.objective)
    else:
        selected_pairings = []
        total_cost = None
    
    return {
        'status': status,
        'selected_pairings': selected_pairings,
        'total_cost': total_cost,
        'solve_time': solve_time,
        'num_pairings_selected': len(selected_pairings)
    }

print("MILP Solver function defined successfully!")

MILP Solver function defined successfully!


In [7]:
# Cell 7: Ensure All Flights Have Coverage - Add Single-Flight Pairings

def ensure_full_coverage(flights_df, pairings):
    """
    Ensure every flight has at least one pairing covering it.
    Add single-flight 'deadhead' pairings for uncovered flights.
    """
    # Build coverage check
    covered_flights = set()
    for p in pairings:
        covered_flights.update(p['flight_ids'])
    
    all_flights = set(flights_df['flight_id'].tolist())
    uncovered = all_flights - covered_flights
    
    new_pairings = list(pairings)
    next_id = max(p['pairing_id'] for p in pairings) + 1 if pairings else 0
    
    for flight_id in uncovered:
        flight = flights_df[flights_df['flight_id'] == flight_id].iloc[0]
        # Create a single-flight pairing with higher cost (deadhead penalty)
        cost = 300 + flight['duration_hours'] * 60 + 50  # Higher cost penalty
        
        new_pairings.append({
            'pairing_id': next_id,
            'flight_ids': [flight_id],
            'num_legs': 1,
            'base': flight['origin'],
            'end_location': flight['destination'],
            'total_flying_hours': flight['duration_hours'],
            'cost': round(cost, 2)
        })
        next_id += 1
    
    print(f"Added {len(uncovered)} single-flight pairings for uncovered flights")
    return new_pairings

# Test
test_pairings_full = ensure_full_coverage(test_flights, test_pairings)
A_full, costs_full = build_constraint_matrix(test_flights, test_pairings_full)
coverage_full = A_full.sum(axis=1)
print(f"After ensuring coverage: Min={coverage_full.min()}, Max={coverage_full.max()}")
print(f"Total pairings: {len(test_pairings_full)}")

Added 15 single-flight pairings for uncovered flights
After ensuring coverage: Min=1, Max=10
Total pairings: 45


In [8]:
# Cell 8: Greedy Heuristic for Comparison

def greedy_solve(flights_df, pairings, A, costs):
    """
    Greedy heuristic: Select pairings with best cost-per-new-flight ratio.
    """
    start_time = time.time()
    num_flights = len(flights_df)
    covered = set()
    selected = []
    total_cost = 0
    
    while len(covered) < num_flights:
        best_pairing = None
        best_ratio = float('inf')
        
        for j, pairing in enumerate(pairings):
            if j in selected:
                continue
            
            # Count new flights this pairing would cover
            new_flights = set(pairing['flight_ids']) - covered
            if len(new_flights) == 0:
                continue
            
            # Cost per new flight
            ratio = pairing['cost'] / len(new_flights)
            if ratio < best_ratio:
                best_ratio = ratio
                best_pairing = j
        
        if best_pairing is None:
            break
        
        selected.append(best_pairing)
        covered.update(pairings[best_pairing]['flight_ids'])
        total_cost += pairings[best_pairing]['cost']
    
    solve_time = time.time() - start_time
    
    # Check for over-coverage (flights covered more than once)
    flight_coverage = {}
    for j in selected:
        for fid in pairings[j]['flight_ids']:
            flight_coverage[fid] = flight_coverage.get(fid, 0) + 1
    
    over_covered = sum(1 for c in flight_coverage.values() if c > 1)
    
    return {
        'status': 'Feasible' if len(covered) == num_flights else 'Infeasible',
        'selected_pairings': selected,
        'total_cost': total_cost,
        'solve_time': solve_time,
        'num_pairings_selected': len(selected),
        'flights_over_covered': over_covered
    }

# Test greedy
greedy_result = greedy_solve(test_flights, test_pairings_full, A_full, costs_full)
print(f"Greedy Status: {greedy_result['status']}")
print(f"Greedy Cost: ${greedy_result['total_cost']:.2f}")
print(f"Greedy Pairings: {greedy_result['num_pairings_selected']}")
print(f"Flights over-covered: {greedy_result['flights_over_covered']}")

Greedy Status: Feasible
Greedy Cost: $10671.50
Greedy Pairings: 20
Flights over-covered: 0


In [9]:
# Cell 9: Generate and Solve Main Problem Instance

# Problem parameters
NUM_FLIGHTS = 40  # 30-50 flights as required
NUM_PAIRINGS = 200  # 100-300 pairings as required

print("="*60)
print("AIRLINE CREW PAIRING OPTIMIZATION")
print("="*60)

# Generate flights
print(f"\n1. Generating {NUM_FLIGHTS} flight legs...")
flights_df = generate_flights(NUM_FLIGHTS, seed=42)
print(f"   Generated flights from {flights_df['departure'].min()} to {flights_df['arrival'].max()}")
print(f"   Airports used: {sorted(set(flights_df['origin']) | set(flights_df['destination']))}")

# Generate pairings
print(f"\n2. Generating {NUM_PAIRINGS} crew pairings...")
pairings = generate_pairings(flights_df, num_pairings=NUM_PAIRINGS, seed=42)
print(f"   Generated {len(pairings)} valid pairings")

# Ensure full coverage
print(f"\n3. Ensuring all flights have coverage...")
pairings = ensure_full_coverage(flights_df, pairings)
print(f"   Total pairings after coverage: {len(pairings)}")

# Build constraint matrix
print(f"\n4. Building constraint matrix A...")
A, costs = build_constraint_matrix(flights_df, pairings)
print(f"   Matrix shape: {A.shape} (M flights x N pairings)")
print(f"   Total non-zero entries: {A.sum()}")

# Solve MILP
print(f"\n5. Solving MILP (Set Partitioning Problem)...")
milp_result = solve_crew_pairing(flights_df, pairings, A, costs)
print(f"   Status: {milp_result['status']}")
print(f"   Solve time: {milp_result['solve_time']:.3f} seconds")

# Solve greedy for comparison
print(f"\n6. Solving Greedy heuristic for comparison...")
greedy_result = greedy_solve(flights_df, pairings, A, costs)

# Results comparison
print("\n" + "="*60)
print("RESULTS COMPARISON")
print("="*60)
print(f"{'Metric':<30} {'MILP':>15} {'Greedy':>15}")
print("-"*60)
print(f"{'Total Cost ($)':<30} {milp_result['total_cost']:>15.2f} {greedy_result['total_cost']:>15.2f}")
print(f"{'Pairings Selected':<30} {milp_result['num_pairings_selected']:>15} {greedy_result['num_pairings_selected']:>15}")
print(f"{'Solve Time (s)':<30} {milp_result['solve_time']:>15.3f} {greedy_result['solve_time']:>15.3f}")

if greedy_result['total_cost'] > 0:
    savings = (greedy_result['total_cost'] - milp_result['total_cost']) / greedy_result['total_cost'] * 100
    print(f"{'MILP Savings vs Greedy':<30} {savings:>15.2f}%")

AIRLINE CREW PAIRING OPTIMIZATION

1. Generating 40 flight legs...
   Generated flights from 2024-01-15 06:15:00 to 2024-01-16 01:49:00
   Airports used: ['ATL', 'DEN', 'DFW', 'JFK', 'LAX', 'MIA', 'ORD', 'PHX', 'SEA', 'SFO']

2. Generating 200 crew pairings...
   Generated 200 valid pairings

3. Ensuring all flights have coverage...
Added 30 single-flight pairings for uncovered flights
   Total pairings after coverage: 230

4. Building constraint matrix A...
   Matrix shape: (40, 230) (M flights x N pairings)
   Total non-zero entries: 254

5. Solving MILP (Set Partitioning Problem)...
   Status: Optimal
   Solve time: 0.735 seconds

6. Solving Greedy heuristic for comparison...

RESULTS COMPARISON
Metric                                    MILP          Greedy
------------------------------------------------------------
Total Cost ($)                        20916.10        20916.10
Pairings Selected                           39              39
Solve Time (s)                           0

In [10]:
# Cell 10: Display Flights and Selected Pairings

print("\nFLIGHT SCHEDULE:")
print("="*80)
display_df = flights_df[['flight_id', 'flight_number', 'origin', 'destination', 
                         'departure', 'arrival', 'duration_hours']].copy()
display_df['departure'] = display_df['departure'].dt.strftime('%H:%M')
display_df['arrival'] = display_df['arrival'].dt.strftime('%H:%M')
print(display_df.to_string(index=False))

print("\n\nSELECTED PAIRINGS (MILP Solution):")
print("="*80)
for idx in milp_result['selected_pairings']:
    p = pairings[idx]
    flights_str = ', '.join([f"FL{1000+fid}" for fid in p['flight_ids']])
    print(f"Pairing {p['pairing_id']:3d}: Base={p['base']}, Flights=[{flights_str}], "
          f"{p['num_legs']} legs, {p['total_flying_hours']:.1f}h, ${p['cost']:.0f}")


FLIGHT SCHEDULE:
 flight_id flight_number origin destination departure arrival  duration_hours
         0        FL1000    ORD         ATL     14:15   15:58            1.72
         1        FL1001    DEN         DFW     09:00   10:47            1.79
         2        FL1002    PHX         SFO     07:00   08:48            1.81
         3        FL1003    ORD         LAX     13:00   17:00            4.00
         4        FL1004    MIA         DEN     19:15   23:11            3.94
         5        FL1005    SEA         LAX     06:15   08:40            2.42
         6        FL1006    SFO         JFK     14:15   19:56            5.69
         7        FL1007    DEN         SFO     09:00   11:26            2.44
         8        FL1008    SFO         ORD     17:30   21:42            4.21
         9        FL1009    PHX         LAX     07:45   08:59            1.24
        10        FL1010    MIA         ORD     18:00   20:54            2.91
        11        FL1011    MIA         LAX   

In [20]:
# Cell 11: Visualization 1 - Flight Network Map

def create_network_map(flights_df, pairings, selected_indices):
    """Create a network diagram showing selected flight routes."""
    fig = go.Figure()
    
    # Add airport nodes
    for code, info in AIRPORTS.items():
        marker_size = 20 if info['is_base'] else 12
        marker_color = 'red' if info['is_base'] else 'blue'
        fig.add_trace(go.Scattergeo(
            lon=[info['lon']],
            lat=[info['lat']],
            mode='markers+text',
            marker=dict(size=marker_size, color=marker_color),
            text=[code],
            textposition='top center',
            name=f"{code} ({'Base' if info['is_base'] else 'Spoke'})",
            showlegend=False
        ))
    
    # Add flight routes for selected pairings
    colors = px.colors.qualitative.Set3
    
    for i, idx in enumerate(selected_indices[:20]):  # Limit to 20 for visibility
        pairing = pairings[idx]
        color = colors[i % len(colors)]
        
        for flight_id in pairing['flight_ids']:
            flight = flights_df[flights_df['flight_id'] == flight_id].iloc[0]
            origin = flight['origin']
            dest = flight['destination']
            
            fig.add_trace(go.Scattergeo(
                lon=[AIRPORTS[origin]['lon'], AIRPORTS[dest]['lon']],
                lat=[AIRPORTS[origin]['lat'], AIRPORTS[dest]['lat']],
                mode='lines',
                line=dict(width=2, color=color),
                name=f"P{pairing['pairing_id']}: {origin}->{dest}",
                showlegend=False
            ))
    
    fig.update_layout(
        title='Flight Network - Selected Routes (First 20 Pairings)',
        geo=dict(
            scope='usa',
            showland=True,
            landcolor='rgb(243, 243, 243)',
            countrycolor='rgb(204, 204, 204)',
        ),
        height=500
    )
    return fig

fig_network = create_network_map(flights_df, pairings, milp_result['selected_pairings'])
fig_network.show()

In [12]:
# Cell 12: Visualization 2 - Crew Schedule Gantt Chart

def create_gantt_chart(flights_df, pairings, selected_indices):
    """Create a Gantt chart showing crew schedules."""
    gantt_data = []
    
    for i, idx in enumerate(selected_indices):
        pairing = pairings[idx]
        crew_name = f"Crew {i+1} (P{pairing['pairing_id']})"
        
        for flight_id in pairing['flight_ids']:
            flight = flights_df[flights_df['flight_id'] == flight_id].iloc[0]
            gantt_data.append({
                'Crew': crew_name,
                'Flight': flight['flight_number'],
                'Start': flight['departure'],
                'End': flight['arrival'],
                'Route': f"{flight['origin']}->{flight['destination']}"
            })
    
    gantt_df = pd.DataFrame(gantt_data)
    
    fig = px.timeline(
        gantt_df,
        x_start='Start',
        x_end='End',
        y='Crew',
        color='Route',
        hover_data=['Flight', 'Route'],
        title='Crew Schedule Gantt Chart'
    )
    
    fig.update_layout(
        height=max(400, len(selected_indices) * 25),
        xaxis_title='Time',
        yaxis_title='Crew Assignment'
    )
    
    return fig

fig_gantt = create_gantt_chart(flights_df, pairings, milp_result['selected_pairings'])
fig_gantt.show()

In [13]:
# Cell 13: Visualization 3 - Cost Breakdown by Pairing

def create_cost_breakdown(pairings, selected_indices):
    """Create a bar chart showing cost breakdown by pairing."""
    selected_pairings = [pairings[i] for i in selected_indices]
    
    pairing_ids = [f"P{p['pairing_id']}" for p in selected_pairings]
    costs = [p['cost'] for p in selected_pairings]
    num_legs = [p['num_legs'] for p in selected_pairings]
    flying_hours = [p['total_flying_hours'] for p in selected_pairings]
    
    fig = make_subplots(rows=2, cols=2,
                        subplot_titles=('Cost by Pairing', 'Legs per Pairing',
                                       'Flying Hours per Pairing', 'Cost Distribution'))
    
    # Cost bar chart
    fig.add_trace(go.Bar(x=pairing_ids, y=costs, name='Cost ($)',
                        marker_color='steelblue'), row=1, col=1)
    
    # Legs bar chart
    fig.add_trace(go.Bar(x=pairing_ids, y=num_legs, name='Legs',
                        marker_color='coral'), row=1, col=2)
    
    # Flying hours bar chart
    fig.add_trace(go.Bar(x=pairing_ids, y=flying_hours, name='Hours',
                        marker_color='mediumseagreen'), row=2, col=1)
    
    # Cost histogram
    fig.add_trace(go.Histogram(x=costs, nbinsx=10, name='Cost Dist',
                              marker_color='purple'), row=2, col=2)
    
    fig.update_layout(height=600, title_text='Cost Breakdown Analysis', showlegend=False)
    return fig

fig_cost = create_cost_breakdown(pairings, milp_result['selected_pairings'])
fig_cost.show()

In [14]:
# Cell 14: Visualization 4 - Coverage Matrix Heatmap

def create_coverage_heatmap(flights_df, pairings, A, selected_indices):
    """Create a heatmap showing which pairings cover which flights."""
    # Get only selected pairings
    A_selected = A[:, selected_indices]
    
    # Create labels
    flight_labels = [f"FL{1000+i}" for i in range(len(flights_df))]
    pairing_labels = [f"P{pairings[i]['pairing_id']}" for i in selected_indices]
    
    fig = go.Figure(data=go.Heatmap(
        z=A_selected,
        x=pairing_labels,
        y=flight_labels,
        colorscale=[[0, 'white'], [1, 'darkblue']],
        showscale=True,
        colorbar=dict(title='Covered')
    ))
    
    fig.update_layout(
        title='Coverage Matrix: Which Pairings Cover Which Flights',
        xaxis_title='Selected Pairings',
        yaxis_title='Flights',
        height=max(400, len(flights_df) * 15),
        width=max(600, len(selected_indices) * 30)
    )
    
    return fig

fig_heatmap = create_coverage_heatmap(flights_df, pairings, A, milp_result['selected_pairings'])
fig_heatmap.show()

In [15]:
# Cell 15: Solution Verification

print("\n" + "="*60)
print("SOLUTION VERIFICATION")
print("="*60)

# Verify each flight is covered exactly once
flight_coverage = np.zeros(len(flights_df), dtype=int)
for idx in milp_result['selected_pairings']:
    for flight_id in pairings[idx]['flight_ids']:
        flight_coverage[flight_id] += 1

print(f"\n1. Flight Coverage Check:")
print(f"   - All flights covered exactly once: {np.all(flight_coverage == 1)}")
print(f"   - Min coverage: {flight_coverage.min()}")
print(f"   - Max coverage: {flight_coverage.max()}")

if not np.all(flight_coverage == 1):
    uncovered = np.where(flight_coverage == 0)[0]
    overcovered = np.where(flight_coverage > 1)[0]
    if len(uncovered) > 0:
        print(f"   - Uncovered flights: {list(uncovered)}")
    if len(overcovered) > 0:
        print(f"   - Over-covered flights: {list(overcovered)}")

# Calculate statistics
selected = [pairings[i] for i in milp_result['selected_pairings']]
total_legs = sum(p['num_legs'] for p in selected)
total_hours = sum(p['total_flying_hours'] for p in selected)
avg_legs = np.mean([p['num_legs'] for p in selected])
avg_hours = np.mean([p['total_flying_hours'] for p in selected])

print(f"\n2. Solution Statistics:")
print(f"   - Total pairings selected: {len(selected)}")
print(f"   - Total flight legs covered: {total_legs}")
print(f"   - Total flying hours: {total_hours:.1f}")
print(f"   - Average legs per pairing: {avg_legs:.2f}")
print(f"   - Average hours per pairing: {avg_hours:.2f}")

# Base distribution
base_counts = {}
for p in selected:
    base_counts[p['base']] = base_counts.get(p['base'], 0) + 1

print(f"\n3. Crew Base Distribution:")
for base, count in sorted(base_counts.items()):
    print(f"   - {base}: {count} pairings")


SOLUTION VERIFICATION

1. Flight Coverage Check:
   - All flights covered exactly once: True
   - Min coverage: 1
   - Max coverage: 1

2. Solution Statistics:
   - Total pairings selected: 39
   - Total flight legs covered: 40
   - Total flying hours: 129.1
   - Average legs per pairing: 1.03
   - Average hours per pairing: 3.31

3. Crew Base Distribution:
   - ATL: 2 pairings
   - DEN: 3 pairings
   - DFW: 2 pairings
   - JFK: 5 pairings
   - LAX: 2 pairings
   - MIA: 6 pairings
   - ORD: 6 pairings
   - PHX: 5 pairings
   - SEA: 4 pairings
   - SFO: 4 pairings


In [16]:
# Cell 16: Multiple Instance Testing (20-30 instances)

print("\n" + "="*80)
print("MULTIPLE INSTANCE TESTING")
print("="*80)

# Define test configurations
test_configs = [
    # (num_flights, num_pairings, cost_multiplier, seed)
    (30, 100, 1.0, 1), (30, 150, 1.0, 2), (30, 200, 1.0, 3),
    (35, 120, 1.0, 4), (35, 180, 1.0, 5), (35, 220, 1.0, 6),
    (40, 150, 1.0, 7), (40, 200, 1.0, 8), (40, 250, 1.0, 9),
    (45, 180, 1.0, 10), (45, 220, 1.0, 11), (45, 280, 1.0, 12),
    (50, 200, 1.0, 13), (50, 250, 1.0, 14), (50, 300, 1.0, 15),
    (35, 150, 0.8, 16), (35, 150, 1.2, 17), (35, 150, 1.5, 18),
    (40, 180, 0.8, 19), (40, 180, 1.2, 20), (40, 180, 1.5, 21),
    (32, 140, 1.0, 22), (38, 170, 1.0, 23), (42, 210, 1.0, 24),
    (48, 240, 1.0, 25), (33, 160, 1.1, 26), (37, 190, 0.9, 27),
]

results = []

print(f"\nRunning {len(test_configs)} test instances...\n")
print(f"{'Instance':<10} {'Flights':<8} {'Pairings':<10} {'MILP Cost':<12} {'Greedy Cost':<12} "
      f"{'Savings %':<10} {'MILP Time':<10} {'Status':<10}")
print("-"*90)

for i, (n_flights, n_pairings, cost_mult, seed) in enumerate(test_configs):
    # Generate instance
    inst_flights = generate_flights(n_flights, seed=seed)
    inst_pairings = generate_pairings(inst_flights, num_pairings=n_pairings, seed=seed)
    
    # Apply cost multiplier
    for p in inst_pairings:
        p['cost'] = p['cost'] * cost_mult
    
    inst_pairings = ensure_full_coverage(inst_flights, inst_pairings)
    inst_A, inst_costs = build_constraint_matrix(inst_flights, inst_pairings)
    
    # Solve
    milp_res = solve_crew_pairing(inst_flights, inst_pairings, inst_A, inst_costs, time_limit=60)
    greedy_res = greedy_solve(inst_flights, inst_pairings, inst_A, inst_costs)
    
    # Calculate savings
    if milp_res['status'] == 'Optimal' and greedy_res['total_cost'] > 0:
        savings = (greedy_res['total_cost'] - milp_res['total_cost']) / greedy_res['total_cost'] * 100
    else:
        savings = 0
    
    results.append({
        'instance': i+1,
        'num_flights': n_flights,
        'num_pairings': len(inst_pairings),
        'milp_cost': milp_res['total_cost'],
        'greedy_cost': greedy_res['total_cost'],
        'savings_pct': savings,
        'milp_time': milp_res['solve_time'],
        'status': milp_res['status']
    })
    
    print(f"{i+1:<10} {n_flights:<8} {len(inst_pairings):<10} "
          f"{milp_res['total_cost'] or 0:<12.2f} {greedy_res['total_cost']:<12.2f} "
          f"{savings:<10.2f} {milp_res['solve_time']:<10.3f} {milp_res['status']:<10}")


MULTIPLE INSTANCE TESTING

Running 27 test instances...

Instance   Flights  Pairings   MILP Cost    Greedy Cost  Savings %  MILP Time  Status    
------------------------------------------------------------------------------------------
Added 22 single-flight pairings for uncovered flights
1          30       122        14858.50     14858.50     0.00       0.033      Optimal   
Added 23 single-flight pairings for uncovered flights
2          30       173        15039.30     15039.30     0.00       0.030      Optimal   
Added 21 single-flight pairings for uncovered flights
3          30       221        15016.10     15016.10     0.00       0.026      Optimal   
Added 21 single-flight pairings for uncovered flights
4          35       141        17564.60     17764.60     1.13       0.023      Optimal   
Added 27 single-flight pairings for uncovered flights
5          35       207        18193.20     18393.20     1.09       0.032      Optimal   
Added 30 single-flight pairings for uncov

In [17]:
# Cell 17: Multi-Instance Summary and Visualization

results_df = pd.DataFrame(results)

print("\n" + "="*60)
print("SUMMARY STATISTICS")
print("="*60)

optimal_results = results_df[results_df['status'] == 'Optimal']
print(f"\nInstances solved optimally: {len(optimal_results)}/{len(results_df)}")
print(f"\nSolve Time Statistics:")
print(f"  - Mean: {optimal_results['milp_time'].mean():.3f}s")
print(f"  - Median: {optimal_results['milp_time'].median():.3f}s")
print(f"  - Max: {optimal_results['milp_time'].max():.3f}s")
print(f"  - Min: {optimal_results['milp_time'].min():.3f}s")

print(f"\nMILP vs Greedy Savings:")
print(f"  - Mean savings: {optimal_results['savings_pct'].mean():.2f}%")
print(f"  - Max savings: {optimal_results['savings_pct'].max():.2f}%")

print(f"\nCost Statistics:")
print(f"  - Mean MILP cost: ${optimal_results['milp_cost'].mean():.2f}")
print(f"  - Mean Greedy cost: ${optimal_results['greedy_cost'].mean():.2f}")


SUMMARY STATISTICS

Instances solved optimally: 19/27

Solve Time Statistics:
  - Mean: 0.030s
  - Median: 0.029s
  - Max: 0.050s
  - Min: 0.023s

MILP vs Greedy Savings:
  - Mean savings: 0.27%
  - Max savings: 1.35%

Cost Statistics:
  - Mean MILP cost: $18953.48
  - Mean Greedy cost: $19011.29


In [18]:
# Cell 18: Final Visualization of Multi-Instance Results

fig = make_subplots(rows=2, cols=2,
                    subplot_titles=('MILP vs Greedy Cost', 'Solve Time vs Problem Size',
                                   'Savings Distribution', 'Cost vs Number of Flights'))

# MILP vs Greedy cost
fig.add_trace(go.Scatter(x=results_df['instance'], y=results_df['milp_cost'],
                        mode='lines+markers', name='MILP Cost'), row=1, col=1)
fig.add_trace(go.Scatter(x=results_df['instance'], y=results_df['greedy_cost'],
                        mode='lines+markers', name='Greedy Cost'), row=1, col=1)

# Solve time vs problem size
fig.add_trace(go.Scatter(x=results_df['num_pairings'], y=results_df['milp_time'],
                        mode='markers', name='Solve Time',
                        marker=dict(size=10, color=results_df['num_flights'],
                                   colorscale='Viridis', showscale=True)), row=1, col=2)

# Savings histogram
fig.add_trace(go.Histogram(x=results_df['savings_pct'], nbinsx=15,
                          name='Savings %', marker_color='green'), row=2, col=1)

# Cost vs flights
fig.add_trace(go.Scatter(x=results_df['num_flights'], y=results_df['milp_cost'],
                        mode='markers', name='Cost vs Flights',
                        marker=dict(size=12)), row=2, col=2)

fig.update_layout(height=700, title_text='Multi-Instance Analysis Results', showlegend=True)
fig.show()

print("\n" + "="*60)
print("ANALYSIS COMPLETE")
print("="*60)


ANALYSIS COMPLETE
